In [ ]:
import yfinance as yf

df = yf.download("^NSEI", start="2018-01-01", end="2019-12-30")

df.columns = df.columns.get_level_values(0)   # remove multi-index
df.head()
df.shape



/tmp/ipython-input-3589808078.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("^NSEI", start="2018-01-01", end="2019-12-30")
[*********************100%***********************]  1 of 1 completed


(484, 5)

In [ ]:
import yfinance as yf

df = yf.download("^NSEI", start="2018-01-01", end="2019-12-30")

df.columns = df.columns.get_level_values(0)   # remove multi-index
df.head()
df.shape



/tmp/ipython-input-3589808078.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("^NSEI", start="2018-01-01", end="2019-12-30")
[*********************100%***********************]  1 of 1 completed


(484, 5)

In [ ]:
!pip install ta
import ta

# RSI
df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

# MACD
macd = ta.trend.MACD(df['Close'])
df['MACD'] = macd.macd()
df['MACD_signal'] = macd.macd_signal()
df['MACD_diff'] = macd.macd_diff()

# Moving averages
df['MA_20'] = df['Close'].rolling(window=20).mean()
df['MA_50'] = df['Close'].rolling(window=50).mean()

df = df.dropna()
df.head()
df.shape


(435, 11)

In [ ]:
import pandas as pd

news_df = pd.read_csv("IndianFinancialNews.csv")
news_df.head()
news_df.shape


(50000, 4)

In [ ]:
news_df['Date'] = pd.to_datetime(news_df['Date'], errors='coerce')
news_df = news_df.dropna(subset=['Date'])
news_df.head()


,Unnamed: 0,Date,Title,Description
0,0,2020-05-26,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,..."
1,1,2020-05-26,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex..."
2,2,2020-05-25,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...
3,3,2020-05-24,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...
4,4,2020-05-23,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...


In [ ]:
news_df = news_df[
    (news_df['Date'] >= "2018-01-01") &
    (news_df['Date'] <= "2019-12-30")
]
news_df.shape


(3920, 4)

In [ ]:
news_df = news_df[['Date', 'Title']]
news_df.head()


,Date,Title
806,2019-12-30,"MDR waiver to hit fintechs, payments companies..."
807,2019-12-30,RBI wants urban cooperative banks to focus mai...
808,2019-12-30,Decade in review: How state-owned banks rose t...
809,2019-12-30,"IFIN reports net loss of Rs 13,272 cr in FY19,..."
810,2019-12-30,RBI allows use of UPI and other non-bank payme...


In [ ]:
!pip install transformers torch -q
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0]

    neg = float(probs[0])
    neu = float(probs[1])
    pos = float(probs[2])

    return pos - neg


In [ ]:
news_df['sentiment'] = news_df['Title'].apply(get_sentiment)
news_df.head()
news_df.shape


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(3920, 3)

In [ ]:
daily_sentiment = news_df.groupby("Date")['sentiment'].mean().reset_index()
daily_sentiment.head()
daily_sentiment.shape


(698, 2)

In [ ]:
daily_sentiment.head()

,Date,sentiment
0,2018-01-01,-0.703910
1,2018-01-02,-0.972608
2,2018-01-03,-0.257623
3,2018-01-04,-0.500307
4,2018-01-05,-0.790951


In [ ]:
df = df.reset_index()
df.head()


Price,Date,Close,High,Low,Open,Volume,RSI,MACD,MACD_signal,MACD_diff,MA_20,MA_50
0,2018-03-15,10360.150391,10420.000000,10346.200195,10405.450195,173900,44.667268,-69.441957,-80.869887,11.427930,10397.350000,10599.366035
1,2018-03-16,10195.150391,10346.299805,10180.250000,10345.150391,328500,37.351377,-78.756945,-80.447299,1.690354,10379.832520,10594.425039
2,2018-03-19,10094.250000,10224.549805,10075.299805,10215.349609,210200,33.714806,-93.206532,-82.999145,-10.207386,10361.930029,10587.446035
3,2018-03-20,10124.349609,10155.650391,10049.099609,10051.549805,203000,35.725189,-101.064142,-86.612145,-14.451997,10349.227490,10579.837031
4,2018-03-21,10155.250000,10227.299805,10132.950195,10181.950195,204800,37.810484,-103.603672,-90.010450,-13.593222,10338.969971,10571.765039


In [ ]:
df_merged = df.merge(daily_sentiment, on="Date", how="left")

df_merged['sentiment'] = df_merged['sentiment'].fillna(0)
df_merged.head()
df_merged.shape


(435, 13)

In [ ]:
df_merged['target'] = (df_merged['Close'].shift(-1) > df_merged['Close']).astype(int)

df_merged = df_merged.dropna()
df_merged.shape


(435, 14)

In [ ]:
df_merged

,Date,Close,High,Low,Open,Volume,RSI,MACD,MACD_signal,MACD_diff,MA_20,MA_50,sentiment,target
0,2018-03-15,10360.150391,10420.000000,10346.200195,10405.450195,173900,44.667268,-69.441957,-80.869887,11.427930,10397.350000,10599.366035,-0.228440,0
1,2018-03-16,10195.150391,10346.299805,10180.250000,10345.150391,328500,37.351377,-78.756945,-80.447299,1.690354,10379.832520,10594.425039,-0.684361,0
2,2018-03-19,10094.250000,10224.549805,10075.299805,10215.349609,210200,33.714806,-93.206532,-82.999145,-10.207386,10361.930029,10587.446035,-0.508209,1
3,2018-03-20,10124.349609,10155.650391,10049.099609,10051.549805,203000,35.725189,-101.064142,-86.612145,-14.451997,10349.227490,10579.837031,-0.722709,1
4,2018-03-21,10155.250000,10227.299805,10132.950195,10181.950195,204800,37.810484,-103.603672,-90.010450,-13.593222,10338.969971,10571.765039,-0.429697,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,2019-12-20,12271.799805,12293.900391,12252.750000,12266.450195,810700,67.409409,93.493758,75.233528,18.260230,12058.995117,11864.901992,-0.996296,0
431,2019-12-23,12262.750000,12287.150391,12213.250000,12235.450195,604800,66.601951,98.598933,79.906609,18.692324,12068.445117,11883.890996,-0.830499,0
432,2019-12-24,12214.549805,12283.700195,12202.099609,12269.250000,470300,62.320192,97.630050,83.451297,14.178752,12077.287598,11903.490996,-0.186832,0
433,2019-12-26,12126.549805,12221.549805,12118.849609,12211.849609,520300,55.326778,88.738417,84.508721,4.229696,12078.580078,11919.920996,-0.634501,1


In [ ]:
from sklearn.model_selection import train_test_split

X = df_merged.drop(columns=['Date', 'target'])
y = df_merged['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train.shape, X_test.shape


((348, 12), (87, 12))

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
X_train_scaled.head()

NameError: name 'X_train_scaled' is not defined

In [ ]:
!pip install xgboost -q
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss'
)

xgb.fit(X_train_scaled, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred = xgb.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)

print("XGBoost Accuracy:", acc)
print(classification_report(y_test, y_pred))


XGBoost Accuracy: 0.45977011494252873
              precision    recall  f1-score   support

           0       0.42      0.54      0.47        39
           1       0.51      0.40      0.45        48

    accuracy                           0.46        87
   macro avg       0.47      0.47      0.46        87
weighted avg       0.47      0.46      0.46        87



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, max_depth=6)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)
print("RF Accuracy:", accuracy_score(y_test, rf_pred))


RF Accuracy: 0.4827586206896552
